In [18]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np

text = open('input.txt').read() # doom lyrics
print('corpus length:', len(text))

tokenizer = Tokenizer()

corpus length: 326073


In [2]:
def dataset_preparation(data):
    corpus = data.lower().split("\n")    
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1 
    
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences,   
                          maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    
    return predictors, label, max_sequence_len, total_words

In [22]:
def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    
    model = Sequential()
    model.add(Embedding(total_words, 50, input_length=input_len))
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(75)))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    model.fit(predictors, label, epochs=100, verbose=1)
    
    return model

In [23]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
  
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [24]:
X, Y, max_len, total_words = dataset_preparation(text)
model = create_model(X, Y, max_len, total_words)

Epoch 1/100
1713/1713 [==============================] - 213s 125ms/step - loss: 7.2658
Epoch 2/100
1713/1713 [==============================] - 192s 112ms/step - loss: 6.8363
Epoch 3/100
1713/1713 [==============================] - 190s 111ms/step - loss: 6.5843
Epoch 4/100
1713/1713 [==============================] - 175s 102ms/step - loss: 6.3574
Epoch 5/100
1713/1713 [==============================] - 165s 96ms/step - loss: 6.1585
Epoch 6/100
1713/1713 [==============================] - 164s 96ms/step - loss: 5.9762
Epoch 7/100
1713/1713 [==============================] - 199s 116ms/step - loss: 5.8041
Epoch 8/100
1713/1713 [==============================] - 205s 120ms/step - loss: 5.6358
Epoch 9/100
1713/1713 [==============================] - 218s 127ms/step - loss: 5.4720
Epoch 10/100
1713/1713 [==============================] - 235s 137ms/step - loss: 5.3096
Epoch 11/100
1713/1713 [==============================] - 191s 111ms/step - loss: 5.1476
Epoch 12/100
1713/1713 [========

1713/1713 [==============================] - 203s 119ms/step - loss: 1.3567
Epoch 94/100
1713/1713 [==============================] - 205s 120ms/step - loss: 1.3361
Epoch 95/100
1713/1713 [==============================] - 214s 125ms/step - loss: 1.3340
Epoch 96/100
1713/1713 [==============================] - 211s 123ms/step - loss: 1.3302
Epoch 97/100
1713/1713 [==============================] - 238s 139ms/step - loss: 1.3222
Epoch 98/100
1713/1713 [==============================] - 205s 119ms/step - loss: 1.3054
Epoch 99/100
1713/1713 [==============================] - 228s 133ms/step - loss: 1.3054
Epoch 100/100
1713/1713 [==============================] - 281s 164ms/step - loss: 1.2939


In [12]:
model.save("../models/variable_length_lstm_2.keras")

In [26]:
generate_text("in love with mary jane", 10, max_len, model)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


"in love with mary jane she's my main thing can't hardly have by the top"

In [27]:
generate_text("light the doobie", 10, max_len, model)

'light the doobie til it glow like a ruby on a train for'